In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'new-plant-diseases-dataset' dataset.
Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [3]:
!pip install -q kaggle "tensorflow>=2.17.0" gradio matplotlib

In [15]:
import os

# The 'path' variable holds the correct root directory of the downloaded dataset.
# Based on the diagnostic prints, the 'train' and 'valid' directories are nested within
# a second 'New Plant Diseases Dataset(Augmented)' folder inside the first one.
base_path = os.path.join(path, "New Plant Diseases Dataset(Augmented)", "New Plant Diseases Dataset(Augmented)")

train_dir = os.path.join(base_path, "train")
valid_dir = os.path.join(base_path, "valid")

print("Train path:", train_dir)
print("Valid path:", valid_dir)

Train path: /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
Valid path: /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os # Import os for directory listing

# Diagnostic prints to inspect the filesystem structure
print(f"Attempting to access train_dir: {train_dir}")
print(f"Attempting to access valid_dir: {valid_dir}")

# List contents of the base download path
try:
    print(f"Contents of {path}: {os.listdir(path)}")
except FileNotFoundError:
    print(f"Error: The base path {path} was not found.")

# List contents of the assumed intermediate directory
intermediate_dir_candidate = os.path.join(path, "New Plant Diseases Dataset(Augmented)")
try:
    print(f"Contents of {intermediate_dir_candidate}: {os.listdir(intermediate_dir_candidate)}")
except FileNotFoundError:
    print(f"Error: The intermediate directory {intermediate_dir_candidate} was not found. This is a likely cause of the FileNotFoundError.")

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Attempting to access train_dir: /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
Attempting to access valid_dir: /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid
Contents of /kaggle/input/new-plant-diseases-dataset: ['New Plant Diseases Dataset(Augmented)', 'new plant diseases dataset(augmented)', 'test']
Contents of /kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented): ['New Plant Diseases Dataset(Augmented)']
Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax') # Output layer with number of classes
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,173,862 (42.62 MB)

 Trainable params: 11,173,862 (42.62 MB)

 Non-trainable params: 0 (0.00 B)

The model is now defined and compiled. The next step is to train the model using the `train_data` and validate it using `valid_data`.

In [21]:
# Clone your repo
!git clone https://github.com/Sujal-Shreshya/PlantCare-AI.git

# Move into repo
%cd PlantCare-AI

# Copy your notebook into repo
!cp /Contents/PlantCareAI.ipynb .

# Git config (first time only)
!git config --global user.email "maheshwaritanya@gmail.com"
!git config --global user.name "Tanya-26"

# Add, commit, push
!git add PlantCareAI.ipynb
!git commit -m "Added notebook from Colab"
!git push

Cloning into 'PlantCare-AI'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
Receiving objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
/content/PlantCare-AI/PlantCare-AI/PlantCare-AI
cp: cannot stat '/Contents/PlantCareAI.ipynb': No such file or directory
fatal: pathspec 'PlantCareAI.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [22]:
!git clone https://github.com/Sujal-Shreshya/PlantCare-AI.git
%cd PlantCare-AI

Cloning into 'PlantCare-AI'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 4.56 KiB | 4.56 MiB/s, done.
/content/PlantCare-AI/PlantCare-AI/PlantCare-AI/PlantCare-AI


In [24]:
!cp /content/PlantCare-AI.ipynb .

cp: cannot stat '/content/PlantCare-AI.ipynb': No such file or directory


In [25]:
!ls /content/

{EAD5C87D-7C06-4E43-A39D-11747709B500}.png  PlantCare-AI  sample_data
